# Manufacturing KPI — EDA (Downtime & Performance)

## Objective
Build an end-to-end data pipeline and KPI dashboard to monitor manufacturing performance, with a focus on:
- **Downtime** (minutes) and **Downtime Rate**
- **Run Ratio** (actual run time vs total batch duration)
- **Top downtime reasons** (Pareto / 80-20) for faster root-cause analysis

This notebook covers:
1) Loading and understanding the raw dataset  
2) Data cleaning & transformation (wide → long downtime table)  
3) Building a **fact table** for KPIs (per batch)  
4) Aggregations needed for a Streamlit **overview dashboard**

## Dataset
Source tables (raw CSV with `|` separator):
- `line-productivity.csv`: batch-level production log (date, product, operator, start/end time)
- `line-downtime.csv`: downtime minutes by factor per batch (wide format)
- `products.csv`: product metadata + minimum batch time
- `downtime-factors.csv`: downtime factor mapping (description + operator error flag)
- `metadata.csv`: dataset field descriptions

## Outputs
This notebook generates processed datasets used by the dashboard:
- `data_processed/fact_batches.parquet`  
  Batch-level fact table with:
  - `duration_min`, `downtime_total_min`, `actual_run_min`
  - `downtime_rate`, `run_ratio` (and optional `efficiency`)
- `data_processed/fact_downtime_long.parquet`  
  Long-format downtime events:
  - `Batch`, `Factor`, `Description`, `downtime_min`, `Operator Error`


## Import Library

In [55]:
import pandas as pd
from pathlib import Path

## Load Data

In [56]:
prod = pd.read_csv("data_raw/line-productivity.csv", sep="|")
dt_wide = pd.read_csv("data_raw/line-downtime.csv", sep="|")
products = pd.read_csv("data_raw/products.csv", sep="|")
factors = pd.read_csv("data_raw/downtime-factors.csv", sep="|")

## Data Quality Check

### line productivity

In [57]:
prod.head()

,Date,Product,Batch,Operator,Start Time,End Time
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00


In [58]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        31 non-null     object
 1   Product     31 non-null     object
 2   Batch       31 non-null     int64 
 3   Operator    31 non-null     object
 4   Start Time  31 non-null     object
 5   End Time    31 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.6+ KB


In [59]:
prod.isnull().sum()

,0
Date,0
Product,0
Batch,0
Operator,0
Start Time,0
End Time,0


In [60]:
prod.duplicated().sum()

np.int64(0)

In [61]:
prod.describe()

,Batch
count,31.000000
mean,422127.129032
std,10.947578
min,422111.000000
25%,422118.500000
50%,422126.000000
75%,422133.500000
max,422148.000000


In [62]:
prod_tmp = prod.copy()
prod_tmp["Date"] = prod_tmp["Date"].astype(str).str.strip()
prod_tmp["Start Time"] = prod_tmp["Start Time"].astype(str).str.strip()
prod_tmp["End Time"] = prod_tmp["End Time"].astype(str).str.strip()

prod_tmp["start_dt"] = pd.to_datetime(prod_tmp["Date"] + " " + prod_tmp["Start Time"], errors="coerce")
prod_tmp["end_dt"]   = pd.to_datetime(prod_tmp["Date"] + " " + prod_tmp["End Time"], errors="coerce")

failed_parse = prod_tmp["start_dt"].isna().sum() + prod_tmp["end_dt"].isna().sum()
cross_midnight = ((prod_tmp["end_dt"].notna()) & (prod_tmp["start_dt"].notna()) & (prod_tmp["end_dt"] < prod_tmp["start_dt"])).sum()

print("Failed datetime parses:", int(failed_parse))
print("Cross-midnight cases:", int(cross_midnight))

Failed datetime parses: 1
Cross-midnight cases: 0


### line downtime

In [63]:
dt_wide.head()

,Batch,Factor 1,2,3,4,5,6,7,8,9,10,11,12
0,Batch,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
1,422111,NaN,60.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
2,422112,NaN,20.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN
3,422113,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,422114,NaN,NaN,NaN,25.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
dt_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Batch     39 non-null     object 
 1   Factor 1  1 non-null      float64
 2   2         6 non-null      float64
 3   3         3 non-null      float64
 4   4         10 non-null     float64
 5   5         4 non-null      float64
 6   6         13 non-null     float64
 7   7         12 non-null     float64
 8   8         7 non-null      float64
 9   9         2 non-null      float64
 10  10        4 non-null      float64
 11  11        4 non-null      float64
 12  12        7 non-null      float64
dtypes: float64(12), object(1)
memory usage: 4.1+ KB


In [65]:
dt_wide.duplicated().sum()

np.int64(0)

In [66]:
dt_wide.describe()

,Factor 1,2,3,4,5,6,7,8,9,10,11,12
count,1.0,6.00000,3.000000,10.000000,4.000000,13.00000,12.000000,7.000000,2.000000,4.000000,4.000000,7.000000
mean,1.0,27.00000,15.000000,22.900000,15.500000,26.00000,21.750000,21.857143,13.000000,14.750000,11.000000,12.285714
std,NaN,22.93469,10.440307,9.915756,7.593857,16.45195,7.700354,12.785781,5.656854,6.601767,1.414214,4.750940
min,1.0,2.00000,3.000000,4.000000,5.000000,5.00000,7.000000,7.000000,9.000000,10.000000,10.000000,7.000000
25%,1.0,12.50000,11.500000,20.000000,12.500000,15.00000,15.750000,14.000000,11.000000,10.000000,10.000000,8.500000
50%,1.0,20.00000,20.000000,22.500000,17.500000,23.00000,22.500000,20.000000,13.000000,12.500000,10.500000,12.000000
75%,1.0,42.50000,21.000000,25.000000,20.500000,30.00000,30.000000,27.000000,15.000000,17.250000,11.500000,15.000000
max,1.0,60.00000,22.000000,43.000000,22.000000,60.00000,30.000000,44.000000,17.000000,24.000000,13.000000,20.000000


### products

In [67]:
products.head()

,Product,Flavor,Size,Min batch time
0,OR-600,Orange,600 ml,60
1,LE-600,Lemon lime,600 ml,60
2,CO-600,Cola,600 ml,60
3,DC-600,Diet Cola,600 ml,60
4,CO-2L,Cola,2 L,98


In [68]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Product         5 non-null      object
 1   Flavor          5 non-null      object
 2   Size            5 non-null      object
 3   Min batch time  5 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes


In [69]:
products.duplicated().sum()

np.int64(0)

### downtime factors

In [70]:
factors.head()

,Factor,Description,Operator Error
0,1,Emergency stop,No
1,2,Batch change,Yes
2,3,Labeling error,No
3,4,Inventory shortage,No
4,5,Product spill,Yes


In [71]:
factors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Factor          12 non-null     int64 
 1   Description     12 non-null     object
 2   Operator Error  12 non-null     object
dtypes: int64(1), object(2)
memory usage: 420.0+ bytes


In [72]:
dup_factors = factors["Factor"].astype(str).str.strip().duplicated().sum()
print("Duplicated Factor keys:", int(dup_factors))

Duplicated Factor keys: 0


##Data Cleaning

In [73]:
# Standardize column names
for _df in [prod, dt_wide, products, factors]:
    _df.columns = _df.columns.str.strip()

In [74]:
# drop junk row (Batch == "Batch")
dt_wide = dt_wide[dt_wide["Batch"].astype(str).str.strip().str.lower() != "batch"].copy()

In [75]:
# Chance batch from object to int
dt_wide["Batch"] = pd.to_numeric(dt_wide["Batch"], errors="coerce")
dt_wide = dt_wide.dropna(subset=["Batch"]).copy()
dt_wide["Batch"] = dt_wide["Batch"].astype(int)

In [76]:
factor_cols = [c for c in dt_wide.columns if c != "Batch"]
dt_wide[factor_cols] = dt_wide[factor_cols].apply(pd.to_numeric, errors="coerce").fillna(0.0)

In [77]:
# parse datetimes
prod["Date"] = prod["Date"].astype(str).str.strip()
prod["Start Time"] = prod["Start Time"].astype(str).str.strip()
prod["End Time"] = prod["End Time"].astype(str).str.strip()

prod["start_dt"] = pd.to_datetime(prod["Date"] + " " + prod["Start Time"], errors="coerce")
prod["end_dt"]   = pd.to_datetime(prod["Date"] + " " + prod["End Time"], errors="coerce")

In [78]:
# drop rows that failed parsing
prod = prod.dropna(subset=["start_dt", "end_dt"]).copy()

In [79]:
# duration
prod["duration_min"] = (prod["end_dt"] - prod["start_dt"]).dt.total_seconds() / 60.0

In [80]:
factors["Factor"] = pd.to_numeric(factors["Factor"], errors="coerce")

In [81]:
# detect operator error column
op_col = None
for c in factors.columns:
    if "operator" in c.lower() and "error" in c.lower():
        op_col = c
        break

In [82]:
# unify name for easier downstream usage
if op_col and op_col != "Operator Error":
    factors = factors.rename(columns={op_col: "Operator Error"})

In [83]:
# keep only needed columns
keep_cols = ["Factor", "Description"]
if "Operator Error" in factors.columns:
    keep_cols.append("Operator Error")

## Feature Engineering

In [84]:
#transform downtime dataframe from wide to long
factor_cols = [c for c in dt_wide.columns if c != "Batch"]

dt_long = dt_wide.melt(
    id_vars=["Batch"],
    value_vars=factor_cols,
    var_name="factor_raw",
    value_name="downtime_min"
).copy()

dt_long["downtime_min"] = pd.to_numeric(dt_long["downtime_min"], errors="coerce").fillna(0.0)
dt_long = dt_long[dt_long["downtime_min"] > 0].copy()

# extract factor id
dt_long["Factor"] = dt_long["factor_raw"].astype(str).str.extract(r"(\d+)").astype(int)
dt_long = dt_long.drop(columns=["factor_raw"])

print("dt_long:", dt_long.shape)
dt_long.head()

dt_long: (61, 3)


,Batch,downtime_min,Factor
38,422111,60.0,2
39,422112,20.0,2
40,422113,50.0,2
44,422117,10.0,2
57,422130,20.0,2


In [85]:
#join downtime with factor descriptions
dt_long = dt_long.merge(factors, on="Factor", how="left")
missing_desc = dt_long["Description"].isna().mean()
print(f"Missing Description ratio: {missing_desc:.2%}")

dt_long.head()

Missing Description ratio: 0.00%


,Batch,downtime_min,Factor,Description,Operator Error
0,422111,60.0,2,Batch change,Yes
1,422112,20.0,2,Batch change,Yes
2,422113,50.0,2,Batch change,Yes
3,422117,10.0,2,Batch change,Yes
4,422130,20.0,2,Batch change,Yes


In [86]:
# total downtime per batch
dt_sum = (
    dt_long.groupby("Batch", as_index=False)["downtime_min"]
    .sum()
    .rename(columns={"downtime_min": "downtime_total_min"})
)

dt_sum.head()

,Batch,downtime_total_min
0,422111,75.0
1,422112,40.0
2,422113,50.0
3,422114,40.0
4,422115,24.0


In [87]:
# build batch-level fact table
df = prod.merge(dt_sum, on="Batch", how="left")
df["downtime_total_min"] = df["downtime_total_min"].fillna(0.0)


In [88]:
# join product min batch time
products_for_join = products.rename(columns={"Min batch time": "min_batch_time_min"}).copy()
products_for_join = products_for_join.drop_duplicates(subset=["Product"])

df = df.merge(
    products_for_join[["Product", "min_batch_time_min"]],
    on="Product",
    how="left"
)
df = df.loc[:, ~df.columns.duplicated()].copy()

In [89]:
# KPI fields
df["actual_run_min"] = (df["duration_min"] - df["downtime_total_min"]).clip(lower=1e-6)
df["downtime_rate"]  = df["downtime_total_min"] / df["duration_min"].clip(lower=1e-6)
df["run_ratio"]      = df["actual_run_min"] / df["duration_min"].clip(lower=1e-6)

df.head()

,Date,Product,Batch,Operator,Start Time,End Time,start_dt,end_dt,duration_min,downtime_total_min,min_batch_time_min,actual_run_min,downtime_rate,run_ratio
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00,2024-08-29 11:50:00,2024-08-29 14:05:00,135.0,75.0,60,60.0,0.555556,0.444444
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00,2024-08-29 14:05:00,2024-08-29 15:45:00,100.0,40.0,60,60.0,0.400000,0.600000
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00,2024-08-29 15:45:00,2024-08-29 17:35:00,110.0,50.0,60,60.0,0.454545,0.545455
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00,2024-08-29 17:35:00,2024-08-29 19:15:00,100.0,40.0,60,60.0,0.400000,0.600000
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00,2024-08-29 19:15:00,2024-08-29 20:39:00,84.0,24.0,60,60.0,0.285714,0.714286


In [92]:
# keep tidy columns
fact_batches = df[
    [
        "Date", "Product", "Batch", "Operator",
        "Start Time", "End Time", "start_dt", "end_dt",
        "duration_min", "downtime_total_min", "actual_run_min",
        "downtime_rate", "run_ratio", "min_batch_time_min"
    ]
].copy()

print("fact_batches:", fact_batches.shape)
fact_batches.head()

fact_batches: (30, 14)


,Date,Product,Batch,Operator,Start Time,End Time,start_dt,end_dt,duration_min,downtime_total_min,actual_run_min,downtime_rate,run_ratio,min_batch_time_min
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00,2024-08-29 11:50:00,2024-08-29 14:05:00,135.0,75.0,60.0,0.555556,0.444444,60
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00,2024-08-29 14:05:00,2024-08-29 15:45:00,100.0,40.0,60.0,0.400000,0.600000,60
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00,2024-08-29 15:45:00,2024-08-29 17:35:00,110.0,50.0,60.0,0.454545,0.545455,60
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00,2024-08-29 17:35:00,2024-08-29 19:15:00,100.0,40.0,60.0,0.400000,0.600000,60
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00,2024-08-29 19:15:00,2024-08-29 20:39:00,84.0,24.0,60.0,0.285714,0.714286,60


In [94]:
fact_batches["Date"] = pd.to_datetime(fact_batches["Date"], errors="coerce")

In [95]:
# daily trend
daily = fact_batches.copy()
daily["day"] = daily["Date"].dt.date

agg_daily = daily.groupby("day", as_index=False).agg(
    batches=("Batch", "nunique"),
    total_downtime=("downtime_total_min", "sum"),
    avg_downtime=("downtime_total_min", "mean"),
    avg_downtime_rate=("downtime_rate", "mean"),
    avg_run_ratio=("run_ratio", "mean"),
    avg_duration=("duration_min", "mean"),
).sort_values("day")

In [96]:
# aggregration by operator
agg_by_operator = fact_batches.groupby("Operator", as_index=False).agg(
    batches=("Batch", "nunique"),
    total_downtime=("downtime_total_min", "sum"),
    avg_downtime_rate=("downtime_rate", "mean"),
    avg_run_ratio=("run_ratio", "mean"),
    avg_duration=("duration_min", "mean"),
).sort_values("total_downtime", ascending=False)

In [97]:
# aggregration by product
agg_by_product = fact_batches.groupby("Product", as_index=False).agg(
    batches=("Batch", "nunique"),
    total_downtime=("downtime_total_min", "sum"),
    avg_downtime_rate=("downtime_rate", "mean"),
    avg_run_ratio=("run_ratio", "mean"),
    avg_duration=("duration_min", "mean"),
).sort_values("total_downtime", ascending=False)


In [98]:
agg_daily.head(), agg_by_operator.head(), agg_by_product.head()

(          day  batches  total_downtime  avg_downtime  avg_downtime_rate  \
 0  2024-08-29        7           244.0     34.857143           0.327974   
 1  2024-08-30       12           444.0     37.000000           0.359655   
 2  2024-08-31        7           165.0     23.571429           0.245207   
 3  2024-09-02        4           245.0     61.250000           0.362012   
 
    avg_run_ratio  avg_duration  
 0       0.672026     94.857143  
 1       0.640345     97.000000  
 2       0.754793     83.571429  
 3       0.637988    159.250000  ,
   Operator  batches  total_downtime  avg_downtime_rate  avg_run_ratio  \
 0  Charlie       11           384.0           0.290270       0.709730   
 3      Mac        7           300.0           0.384745       0.615255   
 1      Dee        7           207.0           0.298820       0.701180   
 2   Dennis        5           207.0           0.359651       0.640349   
 
    avg_duration  
 0    105.272727  
 3    102.857143  
 1     89.571429  

In [99]:
#top reasons
top_reasons = (dt_long.groupby("Description", as_index=False)["downtime_min"]
               .sum()
               .sort_values("downtime_min", ascending=False))
top_reasons["cum_pct"] = top_reasons["downtime_min"].cumsum() / top_reasons["downtime_min"].sum()

top_reasons.head(10)


,Description,downtime_min,cum_pct
7,Machine adjustment,332.0,0.239193
8,Machine failure,254.0,0.422190
4,Inventory shortage,225.0,0.584294
0,Batch change,160.0,0.699568
1,Batch coding error,145.0,0.804035
9,Other,74.0,0.857349
10,Product spill,57.0,0.898415
2,Calibration error,49.0,0.933718
6,Labeling error,42.0,0.963977
5,Label switch,33.0,0.987752


## Save Outputs

In [100]:
out = Path("data_processed")
out.mkdir(exist_ok=True)

fact_batches.to_parquet(out/"fact_batches.parquet", index=False)
dt_long.to_parquet(out/"fact_downtime_long.parquet", index=False)

agg_daily.to_parquet(out/"agg_daily.parquet", index=False)
agg_by_operator.to_parquet(out/"agg_by_operator.parquet", index=False)
agg_by_product.to_parquet(out/"agg_by_product.parquet", index=False)
top_reasons.to_parquet(out/"agg_top_reasons.parquet", index=False)

print("Saved parquet files to:", out.resolve())


Saved parquet files to: /content/data_processed


## Summary